<a href="https://colab.research.google.com/github/Esbern/Python-for-Planners/blob/main/project_example/car%20on%20geojson%20simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import folium
import time
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 1  # seconds per update
EPSG_UTM = 25832  # UTM Zone for Denmark
EPSG_WGS = 4326   # WGS84

# Function to load GeoJSON LineString
def load_road_from_geojson(url):
    """Loads a LineString from a GeoJSON file at the given URL."""
    response = requests.get(url)
    geojson_data = response.json()

    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)

# Function to transform coordinates between WGS84 and UTM
def transform_to_utm(line, from_epsg=EPSG_WGS, to_epsg=EPSG_UTM):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)

def transform_to_wgs(point, from_epsg=EPSG_UTM, to_epsg=EPSG_WGS):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)

class Car:
    """A simple car simulation moving along a road."""

    def __init__(self, road: LineString, speed_kmh: float):
        """
        Initializes the car simulation.

        Parameters:
        - road: A Shapely LineString representing the road in UTM coordinates.
        - speed_kmh: Speed of the car in km/h.
        """
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length

    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Advances the car's position based on speed and time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)  # Prevent overshooting

    def get_position(self):
        """Returns the current position of the car as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length

# Main simulation function
def simulate_car_movement(geojson_url, speed_kmh):
    """Simulates a car moving along a road, updating its position in Google Colab."""

    # Load road and convert to UTM
    road_wgs = load_road_from_geojson(geojson_url)
    road_utm = transform_to_utm(road_wgs)

    # Initialize car
    car = Car(road_utm, speed_kmh)

    # Setup Folium map centered at start point
    start_point_wgs = transform_to_wgs(road_utm.interpolate(0))
    m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)

    # Add road to map
    folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

    # Simulation loop
    while not car.has_reached_end():
        car.advance(TIME_STEP)
        car_pos_wgs = transform_to_wgs(car.get_position())

        # Clear previous markers and add new marker
        m = folium.Map(location=[car_pos_wgs.y, car_pos_wgs.x], zoom_start=15)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

        folium.Marker(
            location=[car_pos_wgs.y, car_pos_wgs.x],
            icon=folium.Icon(color="red", icon="car"),
            popup=f"Speed: {speed_kmh} km/h"
        ).add_to(m)

        # Save map to HTML and reload output in Colab
        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        # Print progress
        print(f"Car position: {car.position_m:.2f} meters / {car.road_length:.2f} meters")

        time.sleep(0.5)  # Simulate real-time updates

    print("Car has reached the end of the road.")

# Run simulation
geojson_url = "https://raw.githubusercontent.com/Esbern/Python-for-Planners/refs/heads/main/data/line.geojson"
simulate_car_movement(geojson_url, speed_kmh=120)


Car position: 12200.00 meters / 29292.75 meters


# Simulating a Car on a Road in Python

## Introduction
This guide walks through a Python script that simulates a car moving along a road. The road is defined as a GeoJSON LineString, which is transformed into a UTM coordinate system for accurate distance calculations. The car moves along the road at a given speed, and its position is updated dynamically on a Folium map in Google Colab. The script follows a structured approach with clear modular components for readability and reusability.

## Overview of the Components
The script consists of several key components:

1. **Loading and transforming the road data**: Reads a GeoJSON file and converts coordinates to UTM for accurate distance calculations.
2. **Defining a `Car` class**: Handles the movement of the car along the road based on its speed.
3. **Simulating movement**: Loops through time steps, updating the car's position.
4. **Displaying the simulation in Folium**: Continuously updates a Folium map in Google Colab.

---

## Step 1: Importing Required Libraries
We start by importing the necessary Python libraries:

```python
import json
import requests
import folium
import time
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output
```

- `requests`: Fetches the GeoJSON file from a URL.
- `shapely`: Handles geometric operations on the road.
- `pyproj`: Transforms coordinates between WGS84 and UTM.
- `folium`: Displays an interactive map with the car's movement.
- `IPython.display`: Ensures smooth real-time map updates in Google Colab.

---

## Step 2: Loading and Transforming the Road Data
### **Loading the GeoJSON LineString**
The road is stored in a GeoJSON file. We define a function to fetch and parse this data:

```python
def load_road_from_geojson(url):
    """Loads a LineString from a GeoJSON file at the given URL."""
    response = requests.get(url)
    geojson_data = response.json()
    
    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)
```

- This function downloads the GeoJSON data.
- Extracts the `LineString` geometry containing the road coordinates.
- Returns the road as a **Shapely LineString**.

### **Transforming Coordinates to UTM**
Since we need accurate distance calculations, we convert the road from WGS84 (latitude/longitude) to UTM:

```python
def transform_to_utm(line, from_epsg=4326, to_epsg=25832):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)
```

- Uses `pyproj.Transformer` to convert WGS84 (`EPSG:4326`) to UTM (`EPSG:25832`).
- Applies the transformation to the road `LineString`.

Similarly, we define a function to convert UTM coordinates back to WGS84 for display:

```python
def transform_to_wgs(point, from_epsg=25832, to_epsg=4326):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)
```

---

## Step 3: Defining the `Car` Class
The `Car` class models a vehicle moving along a road.

```python
class Car:
    """A simple car simulation moving along a road."""
    
    def __init__(self, road: LineString, speed_kmh: float):
        """Initializes the car with a road and speed."""
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length
    
    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Moves the car forward by its speed over a time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)

    def get_position(self):
        """Returns the car's current position as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length
```

- The car moves at a speed given in **km/h**, converted to **m/s**.
- It advances along the road in fixed time steps.
- `line_interpolate_point` determines the car's position.

---

## Step 4: Simulating Movement
The main function runs the simulation and updates a Folium map dynamically.

```python
def simulate_car_movement(geojson_url, speed_kmh):
    """Simulates a car moving along a road, updating its position in Google Colab."""
    
    road_wgs = load_road_from_geojson(geojson_url)
    road_utm = transform_to_utm(road_wgs)
    car = Car(road_utm, speed_kmh)
    
    while not car.has_reached_end():
        car.advance(1)
        car_pos_wgs = transform_to_wgs(car.get_position())

        # Update the map dynamically
        m = folium.Map(location=[car_pos_wgs.y, car_pos_wgs.x], zoom_start=15)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)
        folium.Marker(location=[car_pos_wgs.y, car_pos_wgs.x], icon=folium.Icon(color="red", icon="car")).add_to(m)
        
        # Display in Colab
        clear_output(wait=True)
        display(m)
        time.sleep(0.5)
    
    print("Car has reached the end of the road.")
```

- Updates the map dynamically in **Google Colab**.
- Ensures the latest position is shown while removing the previous output.
- Simulates real-time movement with **time delays**.

---

## Step 5: Running the Simulation
Finally, we run the simulation with a speed of 50 km/h:

```python
geojson_url = "https://raw.githubusercontent.com/Esbern/Python-for-Planners/refs/heads/main/data/line.geojson"
simulate_car_movement(geojson_url, speed_kmh=50)
```

---

## Conclusion
This structured approach makes the simulation modular and reusable. The car moves realistically, and Folium provides an interactive visualization. The **Google Colab compatibility** ensures smooth real-time updates. 🚗💨

